In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [1]:

df = pd.read_csv('preprocess_all_good_lm.csv')

In [6]:
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,preprocessed_q1,preprocessed_q2
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what step step guide invest share market india,what step step guide invest share market
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what story kohinoor koh-i-noor diamond,what would happen indian government steal kohi...
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how increase speed internet connection use vpn,how internet speed increase hack dns
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why mentally lonely how solve,find remainder when math 23^ 24 /math divide 2...
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [5]:
for i in range (3, 35005, 1000):
    print('========')
    print(df.question1[i])
    print(df.preprocessed_q1[i])
    print('----')
    print(df.question2[i])
    print(df.preprocessed_q2[i])

Why am I mentally very lonely? How can I solve it?
why mentally lonely how solve
----
Find the remainder when [math]23^{24}[/math] is divided by 24,23?
find remainder when math 23^ 24 /math divide 24 23
Where can I find my mentor?
where find mentor
----
Where can I find a willing mentor?
where find willing mentor
What name will you give to a mathematics education blog?
what name give mathematic education blog
----
What advice will you give to an IIT graduate in Mechanical/Civil Engineering, who strongly wants to pursue a research career in computer science/mathematics, and has completed the basic courses of CS in coursera?
what advice give iit graduate mechanical civil engineering who strongly want pursue research career computer science mathematic complete basic course cs coursera
What type of diets can you follow to lose 5 pounds in 2 weeks?
what type diet follow lose 5 pound 2 week
----
How can I lose 5lbs in 2 weeks?
how lose 5lbs 2 week
What hotel in Saharanpur would be safe for u

In [10]:
def lev_dist(a1, a2):
    """
    weighted Levenshtein distance
    :param a1, sentence
    :param a2 ,sentence
    :return: weighted Levenshtein distance
    """
    source=a1.split()
    target=a2.split()
    if source == target:
        return 0

    # Prepare a matrix
    slen, tlen = len(source), len(target)
    dist = [[0 for i in range(tlen+1)] for x in range(slen+1)]
    for i in range(slen+1):
        dist[i][0] = i
    for j in range(tlen+1):
        dist[0][j] = j

    # Counting distance, here is my function
    for i in range(slen):
        for j in range(tlen):
            cost = 0 if source[i] == target[j] else 1
            dist[i+1][j+1] = min(
                            dist[i][j+1] + 1,   # deletion
                            dist[i+1][j] + 1,   # insertion
                            dist[i][j] + cost   # substitution
                        )
    return (dist[-1][-1])/\
           ((len(source)+len(target))/2)


if __name__ == '__main__':
    df = df.dropna()
    df['lev_dist'] = np.vectorize(lev_dist)(df['preprocessed_q1'], df['preprocessed_q2'])
    df.to_csv('preprocess_all_lev.csv')
    df['lev_pred']=0
    df['lev_pred']=df.lev_dist < 0.35
    df["lev_pred"]=df["lev_pred"].astype(int)
    df['lev_true']=df.lev_pred ==df.is_duplicate
    print(df['lev_true'].value_counts())

True     260834
False    143411
Name: lev_true, dtype: int64


In [11]:
df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate,preprocessed_q1,preprocessed_q2,lev_dist,lev_pred,lev_true
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what step step guide invest share market india,what step step guide invest share market,0.133333,1,False
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what story kohinoor koh-i-noor diamond,what would happen indian government steal kohi...,0.800000,0,True
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how increase speed internet connection use vpn,how internet speed increase hack dns,0.769231,0,True
3,3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why mentally lonely how solve,find remainder when math 23^ 24 /math divide 2...,1.333333,0,True
4,4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water,1.200000,0,True


In [15]:
score = accuracy_score(df.is_duplicate, df.lev_pred) # total accuracy

print('Accuracy only with Levinstein: ', score)

Accuracy only with Levinstein:  0.6452374179025097
